# Josef Stefan (1879): On the Relation between Heat Radiation and Temperature

## Code Preamble

In [1]:
(require '[clojupyter.misc.helper :as helper])
(def _ (helper/add-dependencies '[org.mentat/emmy "0.32.0"]))
(def _ (helper/add-dependencies '[org.scicloj/kindly  "4-beta14"]))

#'user/_

In [2]:
(ns stefan
    (:refer-clojure :exclude [+ - * / zero? compare divide numerator denominator infinite? abs ref partial =])
    (:require [emmy.env :refer :all]
              [clojure.math :as math]
              [scicloj.kindly.v4.kind :as kind]))

nil

In [3]:
(defn n-transpose [m]
  (apply mapv vector m))

(defn round [x dec]
  (double (/ (math/round (* x (expt 10 dec))) (expt 10 dec))))

#'stefan/round

In [4]:
(defn tex [exp & [prefix]]
    (kind/tex (str "$" prefix (->TeX exp) "$")))

#'stefan/tex

## Data and Calculation

For his 1879 analysis, Josef Stefan did not carry out own experiments. He analysed existing data. Here we focus on his analysis of the data by Dulong&Petit from 60 years earlier. ([link to Stefan's original paper](https://books.google.at/books?id=I9sDAAAAIAAJ&pg=PR2&dq=Sitzungsberichte+der+kaiserlichen+akademie+der+Wissenschaften+mathematisch+naturwissenschaftliche+Classe+neunundsiebzigster+band+1879&hl=en&newbks=1&newbks_redir=0&sa=X&ved=2ahUKEwiIttv4vomLAxUPPhAIHdeLIaoQ6AF6BAgGEAI#v=onepage&q=Sitzungsberichte%20der%20kaiserlichen%20akademie%20der%20Wissenschaften%20mathematisch%20naturwissenschaftliche%20Classe%20neunundsiebzigster%20band%201879&f=false) "Sitzungsberichte der Kaiserlichen Akademie der Wissenschaften.Volume 79, Parts 2-3", p391)

Dulong and Petit placed a heated mercury-in-glass thermometer with a spherical bulb centrally in a spherical enclosure held at (Celsius) temperature zero. Then they noted the rate of change of the temperature with time.

In the following I state the D&P values, as given by Stefan based on their publication from 1817.

Temperature in degree Celsius (°C)

In [5]:

(def temper 
    [80 100 120 140 160 180 200 220 240])
(kind/table {:row-vectors [temper]})

80,100,120,140,160,180,200,220,240


Rate of change of the temperature, given in °C per minute.

In [6]:

(def dp-meas 
    [1.74 2.30 3.02 3.88 4.89 6.10 7.40 8.81 10.69])
(kind/table {:row-vectors [dp-meas]})

1.74,2.3,3.02,3.88,4.89,6.1,7.4,8.81,10.69


### The old formula given by Dulong and Petit

The rate of change of temperature, R [°C/min], is given by the following formula:

In [7]:

(defn dpf [A M] 
    (fn [t] 
        (* M (- (expt A t) 1))))

(defn dp-formel [Celsius]
    (let [t Celsius
          A 1.0077
          M 2.02]
         ((dpf A M) t)))

(tex ((dpf 'A 'M) 't) "R(t)=")

* t ... Temperature of mercury in degree Celsius
* A ... Constant, independent of material, given by D&P as 1.0077
* M ... Material dependent number (see text)

According to D&P, the number $A$ is a constant independent of the material while the number M depends on the material. Below are the calculated values as stated by Dulong and Petit. They supposedly used the above formula.

In [8]:

(def dp-theor 
    [1.72 2.33 3.05 3.89 4.87 6.03 7.34 8.89 10.68])
(kind/table {:row-vectors [dp-theor]})

1.72,2.33,3.05,3.89,4.87,6.03,7.34,8.89,10.68


I could not find any value for M in the paper by Stefan, so I chose M to best match the calculation by D&P. That does not work perfetly, but is close enough as shown below. The difference between measurement and calculation is of the order 0.05 °C/min (third column). The last column is my calculation using M=2.02.

In [9]:

(def tb2 
    (map (fn [[T m t]]
                [T m (round (- t m) 2) t (round (dp-formel T) 2)])
              (n-transpose [temper dp-meas dp-theor])))

(kind/table {:column-names ["Temperature | ", "Measurement | ", 
                            "Correction | ", 
                            "D&P Calculation | " , "My Calculation"] 
             :row-vectors tb2})

80,1.74,-0.02,1.72,1.71
100,2.3,0.03,2.33,2.33
120,3.02,0.03,3.05,3.05
140,3.88,0.01,3.89,3.89
160,4.89,-0.02,4.87,4.87
180,6.1,-0.07,6.03,6.01
200,7.4,-0.06,7.34,7.35
220,8.81,0.08,8.89,8.9
240,10.69,-0.01,10.68,10.71


### Stefan's $T^4$ power law

The new formula given by Stefan is:

In [10]:

(defn sf [T B]
    (fn [t]
        (* B (- (expt (+ T t) 4) (expt T 4)))))

(def B (/ 1 (* 6 (expt 10 9))))
(def T 273)

(defn stefan-formel [t] ((sf T B) t))
    
(tex ((sf 'T 'B) 't) "R(t)=")

Above: the famous power law given by Stefan
* t ... Temperature of mercury in degree Celsius
* T ... Number relating 0°C to absolute zero temperature, i.e. plus 273 degrees
* B ... One sixth billion, heuristic factor given by Stefan 

The factors in the D&P formula were close to 1. In contrast, the numbers in the new power law are huge. I suspect this as one reason why noone before Stefan made an attempt. Lets looks at the magnitudes, remember no computers in 1879. Would you do the Stefan formula by hand?

In [11]:

(kind/table 
 {:row-vectors
  [["B" B] ["T^4" (expt T 4)] ["(T + 100)^4" (expt (+ T 100) 4)]]})

B,1/6000000000
T^4,5554571841
(T + 100)^4,19356878641


Below are the calculated values as stated by Stefan. They match exactly my (computer-) calculations using his formula, i.e. subtracting huge numbers to get numbers of magnitude 10.

In [12]:

(def st-theor 
    [1.66 2.30 3.05 3.92 4.93 6.09 7.42 8.92 10.62])
(kind/table {:row-vectors [st-theor]})

1.66,2.3,3.05,3.92,4.93,6.09,7.42,8.92,10.62


Finally, the comparison of Stefan's theoretical values to measurement. The error-margin is the same as for D&P before, of magintude 0.05

In [13]:

(def tb7 
    (map (fn [[T m st-t]]
                [T m (round (- st-t m) 2) st-t
                 (round (stefan-formel T) 2)])
              (n-transpose [temper dp-meas st-theor])))

(kind/table {:column-names ["Temperature | ", "Measurement | ",  
                            "Stafan Calculation | ",
                            "My Calculation"] 
             :row-vectors tb7})


80,1.74,-0.08,1.66,1.66
100,2.3,0.0,2.3,2.3
120,3.02,0.03,3.05,3.05
140,3.88,0.04,3.92,3.92
160,4.89,0.04,4.93,4.93
180,6.1,-0.01,6.09,6.09
200,7.4,0.02,7.42,7.42
220,8.81,0.11,8.92,8.92
240,10.69,-0.07,10.62,10.62


The advantage of this new formula was a more accurate estimation of the temperature of the sun. The prevailing value at that time, the 1500°C calculated with the D&P formula, was already then suspected to be too low. Indeed the old formula had no value for high temperatures. Today's value is 5800°C, a value which is within the new estimations done by Stefan using his formula.